# RNN
## TensorFlow

### Basics

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent = 4)
sess = tf.InteractiveSession()

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

#### hello

In [3]:
with tf.variable_scope('one_cell') as scope:
    # input_dim = 4, output_dim = 2
    hidden_size = 2
    cell = rnn.BasicRNNCell(num_units = hidden_size)
    print(cell.output_size, cell.state_size)
    
    x_data = np.array([[h]], dtype = np.float32)
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

2 2
array([[[1., 0., 0., 0.]]], dtype=float32)
array([[[-0.5765048 , -0.36967593]]], dtype=float32)


In [4]:
with tf.variable_scope('two_sequences') as scope:
    # input_dim = 4, output_dim = 2, sequence = 5
    hidden_size = 2
    cell = rnn.BasicRNNCell(num_units = hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype = np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval)

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
<bound method Tensor.eval of <tf.Tensor 'two_sequences/rnn/transpose_1:0' shape=(1, 5, 2) dtype=float32>>


In [5]:
y_data = tf.constant([[1, 1, 1]])
prediction = tf.constant([[[0.2, 0.7], [0.6, 0.2], [0.2, 0.9]]], dtype=tf.float32)
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=prediction, targets=y_data, weights=weights)
sess.run(tf.global_variables_initializer())
print("Loss: ", sequence_loss.eval())

Loss:  0.5967595


In [6]:
prediction1 = tf.constant([[[0.3, 0.7], [0.3, 0.7], [0.3, 0.7]]], dtype=tf.float32)
prediction2 = tf.constant([[[0.1, 0.9], [0.1, 0.9], [0.1, 0.9]]], dtype=tf.float32)
sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval(),
      "Loss2: ", sequence_loss2.eval())

Loss1:  0.5130153 Loss2:  0.3711007


### Hi Hello : hihell -> ihello

In [2]:
idx2char = ['h', 'i', 'e', 'l', 'o']
x_data = [[0, 1, 0, 2, 3, 3]]
x_one_hot = [[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], 
              [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0]]]
y_data = [[1, 0, 2, 3, 3, 4]]

num_classes = 5
input_dim = 5
hidden_size = 5
batch_size = 1
sequence_length = 6
learning_rate = 0.1

In [3]:
X = tf.placeholder(tf.float32, [None, sequence_length, input_dim])
Y = tf.placeholder(tf.int32, [None, sequence_length])

cell = rnn.BasicLSTMCell(num_units = hidden_size, state_is_tuple = True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state = initial_state,
                                    dtype = tf.float32)

In [15]:
outputs.shape

TensorShape([Dimension(1), Dimension(6), Dimension(5)])

In [4]:
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets = Y, 
                                                weights = weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

In [5]:
prediction = tf.argmax(outputs, axis = 2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict = {X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict = {X: x_one_hot})
        print(i, 'loss: ', l, 'prediction: ', result, 'true Y: ', y_data, end = '')
        
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print('\tPreduction str: ', ''.join(result_str))

0 loss:  1.6004362 prediction:  [[1 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]	Preduction str:  ieelll
1 loss:  1.5065582 prediction:  [[3 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]	Preduction str:  leelll
2 loss:  1.4283648 prediction:  [[3 2 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]	Preduction str:  lellll
3 loss:  1.3643515 prediction:  [[2 2 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]	Preduction str:  eellll
4 loss:  1.3051772 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]	Preduction str:  eeelll
5 loss:  1.2452815 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]	Preduction str:  eeelll
6 loss:  1.1890583 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]	Preduction str:  eeelll
7 loss:  1.1351612 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]	Preduction str:  eeelll
8 loss:  1.0822697 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]	Preduction str:  eeelll
9 loss:  1.0367856 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]	Pred

### Long Sequence RNN

In [2]:
sample = ' if you want you'
idx2char = list(set(sample))
char2idx = {c: i for i, c in enumerate(idx2char)} # char -> idx

dict_size = len(char2idx)
hidden_size = len(char2idx)
num_classes = len(char2idx)
batch_size = 1
sequence_length = len(sample) - 1
learning_rate = 0.1

sample_idx = [char2idx[c] for c in sample]

print(idx2char)
print(char2idx)
print(sample_idx)

['w', 't', 'a', 'f', 'i', 'n', 'o', 'y', ' ', 'u']
{'w': 0, 't': 1, 'a': 2, 'f': 3, 'i': 4, 'n': 5, 'o': 6, 'y': 7, ' ': 8, 'u': 9}
[8, 4, 3, 8, 7, 6, 9, 8, 0, 2, 5, 1, 8, 7, 6, 9]


In [3]:
x_data = [sample_idx[:-1]]
y_data = [sample_idx[1:]]

X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

x_one_hot = tf.one_hot(X, num_classes)
cell = rnn.BasicLSTMCell(num_units = hidden_size, state_is_tuple = True)

In [4]:
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state = initial_state, 
                                     dtype = tf.float32)

X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, 
                                            activation_fn = None)
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(outputs, Y, weights)
loss = tf.reduce_mean(sequence_loss)
trian = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis = 2)

In [5]:
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, 
                                                 weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 2.3026102 Prediction: uuuuuuuuuuuuuuu
1 loss: 2.1664429 Prediction: yyyyyu yyyoyyyu
2 loss: 2.0629306 Prediction: yoyyou      oou
3 loss: 1.7918866 Prediction: yyyyou      oou
4 loss: 1.5047005 Prediction: yyyyou y nnyyou
5 loss: 1.1587769 Prediction: yf you wnnn you
6 loss: 0.78382695 Prediction: yf you want you
7 loss: 0.4591326 Prediction: yf you wan  you
8 loss: 0.3100561 Prediction: if you want you
9 loss: 0.16840306 Prediction: if you want you
10 loss: 0.11590148 Prediction: if you want you
11 loss: 0.05929768 Prediction: if you want you
12 loss: 0.024187865 Prediction: if you want you
13 loss: 0.012777647 Prediction: if you want you
14 loss: 0.007706294 Prediction: if you want you
15 loss: 0.0018703882 Prediction: if you want you
16 loss: 0.0010819497 Prediction: if you want you
17 loss: 0.0016029682 Prediction: if you want you
18 loss: 0.0011642732 Prediction: if you want you
19 loss: 0.0005583805 Prediction: if you want you
20 loss: 0.00034766676 Prediction: if you wan

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
idx2char = list(set(sentence))
char2idx = {c: i for i, c in enumerate(idx2char)} # char -> idx

dict_size = len(char2idx)
hidden_size = len(char2idx)
num_classes = len(char2idx)
batch_size = 1
sequence_length = 10
learning_rate = 0.1

dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    #print(i, x_str, '->', y_str)

    x = [char2idx[c] for c in x_str]  # x str to index
    y = [char2idx[c] for c in y_str]  # y str to index

    dataX.append(x)
    dataY.append(y)

In [3]:
batch_size = len(dataX)

X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

X_one_hot = tf.one_hot(X, num_classes)
X_one_hot.shape

TensorShape([Dimension(None), Dimension(10), Dimension(25)])

In [4]:
cell = rnn.BasicLSTMCell(num_units = hidden_size, state_is_tuple=True)
# how many layers = 2
cell = rnn.MultiRNNCell([cell] * 2, state_is_tuple = True)

initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, initial_state = initial_state, 
                                     dtype = tf.float32)
weights = tf.ones([batch_size, sequence_length])

ouputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

# always - almost like an idiom
X_for_softmax = tf.reshape(outputs, [-1, hidden_size])

softmax_w = tf.get_variable('softmax_w', [hidden_size, num_classes])
softmax_b = tf.get_variable('softmax_b', [num_classes])
outputs = tf.matmul(X_for_softmax, softmax_w) + softmax_b

outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, 
                                                 weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run([train, loss, outputs], feed_dict = {X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis = 1)
    if i % 50 == 0:
        print(i, l)
        
results = sess.run(outputs, feed_dict = {X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis = 1)
    if j is 0: # print all for the first result to make a sentence
        print(''.join([idx2char[t] for t in index]), end = '')
    else:
        print(idx2char[index[-1]], end = '')

0 3.178678
50 0.30942163
100 0.23328196
150 0.23062876
200 0.22993594
250 0.22944461
300 0.22945312
350 0.22966756
400 0.22894523
450 0.22887793
p you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.